In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load c14 and preprocess the data

c14="DDW-0000C-14.xls"
df_c14=pd.read_excel(c14,skiprows=[0,2,3,4,5,6])
df_c14 = df_c14[["State","Area Name","Age-group","Total ","Unnamed: 6","Unnamed: 7"]]
df_c14.columns=["State Code","Name","Age-group","TOT_P","Males","Females"]
df_c14 =df_c14[(df_c14["Age-group"]!="0-4") & (df_c14["Age-group"]!="All ages")]

In [3]:
#merge c14 cols to get the required age groups according to c18
cols_group=["State Code","Name"]
sum_cols={'TOT_P':'sum',
          'Males':'sum',
          'Females':'sum'}
age_30_49=['30-34','35-39','40-44','45-49']
age_50_69=['50-54','55-59','60-64','65-69']
age_70Plus=['70-74','75-79','80+']

In [4]:
#form 30-49 age group by adding the respective smaller groups
df_c14_30_49=df_c14.loc[df_c14['Age-group'].isin(age_30_49)].groupby(cols_group,as_index=False).agg(sum_cols)
df_c14_30_49["Age-group"]="30-49"

In [5]:
#form 50-69 age group by adding the respective smaller groups
df_c14_50_69=df_c14.loc[df_c14['Age-group'].isin(age_50_69)].groupby(cols_group,as_index=False).agg(sum_cols)
df_c14_50_69["Age-group"]="50-69"

In [6]:
#form 70+ age group by adding the respective smaller groups
df_c14_70Plus =df_c14.loc[df_c14['Age-group'].isin(age_70Plus)].groupby(cols_group,as_index=False).agg(sum_cols)
df_c14_70Plus["Age-group"]="70+"

In [7]:
df_sum_merge = pd.concat([df_c14_30_49,df_c14_50_69,df_c14_70Plus])
df_sum_merge.sort_values(by=["State Code","Age-group"],ignore_index=True,inplace=True)

In [8]:
df_c14=df_c14.loc[df_c14['Age-group'].isin(['5-9','10-14','15-19','20-24','25-29','Age not stated'])]

In [9]:
#concat all the age groups dataframes for single df of age groups
df_c14=pd.concat([df_c14,df_sum_merge])

In [10]:
df_c14.sort_values(by=["State Code","Age-group"],inplace=True,ignore_index=True)
del df_c14['Name']
df_c14

,State Code,Age-group,TOT_P,Males,Females
0,0,10-14,132709212,69418835,63290377
1,0,15-19,120526449,63982396,56544053
2,0,20-24,111424222,57584693,53839529
3,0,25-29,101413965,51344208,50069757
4,0,30-49,308492074,157263555,151228519
...,...,...,...,...,...
319,35,30-49,119161,64612,54549
320,35,5-9,30454,15512,14942
321,35,50-69,47692,26817,20875
322,35,70+,8822,4952,3870


In [11]:
#load c18
c18="DDW-C18-0000.xlsx"
cols=["State","Area Name","Total/","Age-group","Unnamed: 6","Unnamed: 7","Unnamed: 9","Unnamed: 10"]
df_c18=pd.read_excel(c18,usecols=cols,skiprows=[0,4,5])

#preprocess c18
df_c18=df_c18[(df_c18["Total/"]=="Total")&(df_c18["Age-group"]!="Total")]
df_c18=df_c18.drop(["Total/"],axis=1).reset_index(drop=True)
df_c18.columns=["State Code","Area Name","Age-group","Males_2","Females_2","Males_3","Females_3"]
df_c18 

,State Code,Area Name,Age-group,Males_2,Females_2,Males_3,Females_3
0,00,INDIA,5-9,8166843,7482349,978151,865957
1,00,INDIA,10-14,18133423,16355069,3831131,3423204
2,00,INDIA,15-19,22750908,19673691,6792766,5833951
3,00,INDIA,20-24,22386694,18957712,7067614,5766720
4,00,INDIA,25-29,19695033,16229012,6144045,4713126
...,...,...,...,...,...,...,...
319,35,ANDAMAN & NICOBAR ISLANDS,25-29,17622,14987,5320,4396
320,35,ANDAMAN & NICOBAR ISLANDS,30-49,51816,40087,16469,10611
321,35,ANDAMAN & NICOBAR ISLANDS,50-69,20054,12303,5517,2286
322,35,ANDAMAN & NICOBAR ISLANDS,70+,3115,1606,579,192


In [12]:
df_c14["State Code"]=df_c14["State Code"].astype(int)
df_c18["State Code"]=df_c18["State Code"].astype(int)

In [13]:
#merge c18 and c14
df_merge = pd.merge(df_c18,df_c14,how='inner',left_on=['State Code','Age-group'],right_on = ['State Code','Age-group'])

#cal one lang and two lang speakers of males
df_merge["Males_L1"]=df_merge["Males"]-df_merge["Males_2"]
df_merge["Males_L2"]=df_merge["Males_2"]-df_merge["Males_3"]

#cal one lang and two lang speakers of females
df_merge["Females_L1"]=df_merge["Females"]-df_merge["Females_2"]
df_merge["Females_L2"]=df_merge["Females_2"]-df_merge["Females_3"]

df_merge.rename(columns={'Males_3':'Males_L3','Females_3':'Females_L3'},inplace=True)
df_merge.drop(['Males_2','Females_2'],axis=1,inplace=True)

In [14]:
#cal ratio of males speaking 3 lang,2 lang and one lang
df_merge["males-ratio-of-3"]=((df_merge["Males_L3"])/df_merge["Males"]).astype(float)
df_merge["males-ratio-of-2"]=((df_merge["Males_L2"])/df_merge["Males"]).astype(float)
df_merge["males-ratio-of-1"]=((df_merge["Males_L1"])/df_merge["Males"]).astype(float)

#cal ratio of females speaking 3 lang,2 lang and one lang
df_merge["females-ratio-of-3"]=((df_merge["Females_L3"])/df_merge["Females"]).astype(float)
df_merge["females-ratio-of-2"]=((df_merge["Females_L2"])/df_merge["Females"]).astype(float)
df_merge["females-ratio-of-1"]=((df_merge["Females_L1"])/df_merge["Females"]).astype(float)
df_merge = df_merge[['State Code','Age-group','males-ratio-of-3','females-ratio-of-3','males-ratio-of-2','females-ratio-of-2','males-ratio-of-1','females-ratio-of-1']]

In [15]:
#separating male and female data into different data frames and sorting by ratios

df_3_m=df_merge[['State Code','Age-group','males-ratio-of-3']]
df_3_m.sort_values(by=['State Code','males-ratio-of-3'],ignore_index=True,inplace=True)

df_3_f=df_merge[['State Code','Age-group','females-ratio-of-3']]
df_3_f.sort_values(by=['State Code','females-ratio-of-3'],ignore_index=True,inplace=True)

df_2_m=df_merge[['State Code','Age-group','males-ratio-of-2']]
df_2_m.sort_values(by=['State Code','males-ratio-of-2'],ignore_index=True,inplace=True)

df_2_f=df_merge[['State Code','Age-group','females-ratio-of-2']]
df_2_f.sort_values(by=['State Code','females-ratio-of-2'],ignore_index=True,inplace=True)

df_1_m=df_merge[['State Code','Age-group','males-ratio-of-1']]
df_1_m.sort_values(by=['State Code','males-ratio-of-1'],ignore_index=True,inplace=True)

df_1_f=df_merge[['State Code','Age-group','females-ratio-of-1']]
df_1_f.sort_values(by=['State Code','females-ratio-of-1'],ignore_index=True,inplace=True)               

In [16]:
#selecting highest ratio in each of the data frames created above
df_3=df_3_m[8: :9]
df_3["females-age-group"]=df_3_f[8: :9]["Age-group"]
df_3["females-ratio-of-3"] = df_3_f[8: :9]["females-ratio-of-3"]
df_3.rename(columns={'State Code':'state/ut','Age-group':'males-age-group'},inplace=True)

df_2=df_2_m[8: :9]
df_2["females-age-group"]=df_2_f[8: :9]["Age-group"]
df_2["females-ratio-of-2"] = df_2_f[8: :9]["females-ratio-of-2"]
df_2.rename(columns={'State Code':'state/ut','Age-group':'males-age-group'},inplace=True)

df_1=df_1_m[8: :9]
df_1["females-age-group"]=df_1_f[8: :9]["Age-group"]
df_1["females-ratio-of-1"] = df_1_f[8: :9]["females-ratio-of-1"]
df_1.rename(columns={'State Code':'state/ut','Age-group':'males-age-group'},inplace=True)

In [17]:
df_3_m[df_3_m["State Code"]==3]

,State Code,Age-group,males-ratio-of-3
27,3,5-9,0.078215
28,3,70+,0.177782
29,3,Age not stated,0.227056
30,3,10-14,0.282455
31,3,50-69,0.292463
32,3,30-49,0.341864
33,3,25-29,0.407402
34,3,20-24,0.445866
35,3,15-19,0.446176


In [18]:
df_2

,state/ut,males-age-group,males-ratio-of-2,females-age-group,females-ratio-of-2
8,0,20-24,0.266027,20-24,0.245006
17,1,10-14,0.491022,10-14,0.473909
26,2,25-29,0.168465,20-24,0.153183
35,3,10-14,0.264767,10-14,0.265602
44,4,30-49,0.289201,70+,0.305369
53,5,20-24,0.242618,20-24,0.203420
62,6,20-24,0.259946,20-24,0.243538
71,7,20-24,0.434504,20-24,0.432642
80,8,20-24,0.144275,15-19,0.121282
89,9,20-24,0.178002,20-24,0.155031


In [19]:
#rename the names of columns
df_1.columns=["state/ut","age-group-males","ratio-males","age-group-females","ratio-females"]
df_2.columns=["state/ut","age-group-males","ratio-males","age-group-females","ratio-females"]
df_3.columns=["state/ut","age-group-males","ratio-males","age-group-females","ratio-females"]

In [20]:
#adjusting the csv view setting
df_1= df_1.replace({'5-9':'5-9'+'\t','10-14':'10-14'+'\t'})
df_2 = df_2.replace({'5-9':'5-9'+'\t','10-14':'10-14'+'\t'})
df_3 = df_3.replace({'5-9':'5-9'+'\t','10-14':'10-14'+'\t'})

In [21]:
outname_1 = 'age-gender-c.csv'
outname_2 = 'age-gender-b.csv'
outname_3 = 'age-gender-a.csv'

outfiles = './Output_Files'
if not os.path.exists(outfiles):
    os.mkdir(outfiles)
outdir=outfiles+'/Q8'
if not os.path.exists(outdir):
    os.mkdir(outdir)
filepath_1 = os.path.join(outdir, outname_1)
filepath_2 = os.path.join(outdir, outname_2)
filepath_3 = os.path.join(outdir, outname_3)

df_1.to_csv(filepath_1,index=False)
df_2.to_csv(filepath_2,index=False)
df_3.to_csv(filepath_3,index=False)